# Figure 2

In [ ]:
using DifferentialEquations
using ModelingToolkit
using MitochondrialDynamics
using ThreadsX
import MitochondrialDynamics: GlcConst
import MitochondrialDynamics.Utils: second, μM, mV, mM, Hz
import PyPlot as plt
rcParams = plt.PyDict(plt.matplotlib."rcParams")
rcParams["font.size"] = 14
rcParams["font.sans-serif"] = "Arial"
rcParams["font.family"] = "sans-serif"

In [ ]:
glc = range(3.0mM, 30.0mM, length=101)  # Range of glucose

@named sys = make_model()
idxGlc = findfirst(isequal(GlcConst), parameters(sys))
prob = SteadyStateProblem(sys, DEFAULT_U0)

In [ ]:
sols = ThreadsX.map(glc) do g
    params = copy(prob.p)
    params[idxGlc] = g
    sol = solve(remake(prob, p=params), DynamicSS(Rodas5()))
end

In [ ]:
import MitochondrialDynamics: G3P, Pyr, NADH_c, NADH_m, ATP_c, ADP_c, AMP_c, Ca_m, Ca_c, x, ΔΨm, degavg

"Plot figure 2"
function plot_fig2(sols, glc; size=(20,15),
                   glcscale = 5mM,
                   xlabelfontsize=16)
    
    extract(sols, k) = map(s->s[k], sols)
    
    # Plot
    glc5 = glc ./ glcscale

    fig, ax = plt.subplots(3, 3, figsize=size)

    ax[1, 1].plot(glc5, extract(sols, G3P) .* 1000 )
    ax[1, 1].set(title="(A) G3P (μM)", ylim=(0.0, 10.0))

    ax[1, 2].plot(glc5, extract(sols, Pyr) .* 1000 )
    ax[1, 2].set(title="(B) Pyruvate (μM)", ylim=(0.0, 125.0))

    ax[1, 3].plot(glc5, extract(sols, Ca_c) .* 1000, label="cyto")
    ax[1, 3].plot(glc5, extract(sols, Ca_m) .* 1000, label="mito")
    ax[1, 3].set(ylim=(0.0, 1.5), title="(C) Calcium (μM)")
    ax[1, 3].legend(loc="right")

    ax[2, 1].plot(glc5, extract(sols, NADH_c) .* 1000, label="cyto")
    ax[2, 1].plot(glc5, extract(sols, NADH_m) .* 1000, label="mito")
    ax[2, 1].set(title="(D) NADH (μM)")
    ax[2, 1].legend(loc="right")

    ax[2, 2].plot(glc5, extract(sols, ATP_c) .* 1000, label="ATP")
    ax[2, 2].plot(glc5, extract(sols, ADP_c) .* 1000, label="ADP")
    ax[2, 2].plot(glc5, extract(sols, AMP_c) .* 1000, label="AMP")
    ax[2, 2].set(title="(E) Adenylates (μM)")
    ax[2, 2].legend(loc="right")

    ax[2, 3].plot(glc5, extract(sols, ATP_c) ./ extract(sols, ADP_c))
    ax[2, 3].set(title="(F) ATP/ADP ratio" , ylim=(0.0, 45.0))

    ax[3, 1].plot(glc5, extract(sols, ΔΨm) .* 1000)
    ax[3, 1].set(title="(G) ΔΨ (mV)", ylim=(80, 160))
    ax[3, 1].set_xlabel("Glucose (X)", fontsize=xlabelfontsize)
    # ax[3, 1].legend(loc="best")

    ax[3, 2].plot(glc5, extract(sols, x[1]), label="X1")
    ax[3, 2].plot(glc5, extract(sols, x[2]), label="X2")
    ax[3, 2].plot(glc5, extract(sols, x[3]), label="X3")
    ax[3, 2].set_title("(H) Mitochondrial nodes")
    ax[3, 2].set_xlabel("Glucose (X)", fontsize=xlabelfontsize)
    ax[3, 2].legend()

    ax[3, 3].plot(glc5, extract(sols, degavg))
    ax[3, 3].set_title("(I) Average Node Degree")
    ax[3, 3].set_xlabel("Glucose (X)", fontsize=xlabelfontsize)

    plt.tight_layout()
    return fig
end

In [ ]:
fig2 = plot_fig2(sols, glc, size=(12, 8));

In [ ]:
# Uncomment if pdf file is required
# fig2.savefig("Fig2.pdf")